In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [58]:
# Libraries Import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


# Setting views


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [59]:
# !pip install pycaret -q
# from pycaret.anomaly import *

## Data Load

In [60]:
data = pd.read_csv('/content/drive/MyDrive/PG Thappar - CSE/Data Science Foundation/DSF-Labs/Real-Estate/Data/Gurgaon_property_data_final_v0_for_EDA.csv')
data.head(2)

,property_name,property_type,society,price,area,bedRoom,bathroom,balcony,additionalRoom,address,floorNum,facing,airport,station,hospitals,banks,malls,educational_institutes,has_lift,has_swimming_pool,has_security_alarm,has_gym,has_park,has_parking,has_water_storage,status,avgRating,Modular Kitchen,Bed,Washing Machine,TV,Stove,Sofa,Fridge,Microwave,Dining Table,Water Purifier,Wardrobe,Curtains,Chimney,AC,Light,Geyser,Exhaust Fan,Fan,sector
0,3 BHK Flat in Sector 36 Sohna,flat,signature global park 4,0.82,1081.0,3,2,2,not available,"144, Sector 36 Sohna, Gurgaon, Haryana",2.0,NaN,True,True,1.0,0.0,2.0,3.0,False,False,False,False,True,False,False,constructed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36
1,2 BHK Flat in Sector 89 Gurgaon,flat,smart world gems,0.95,1105.0,2,2,2,"study room,servant room","O 118d, Sector 89 Gurgaon, Gurgaon, Haryana",4.0,NaN,False,True,1.0,0.0,1.0,2.0,False,True,False,True,True,False,False,underConstruction,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89


## Preprocessing

In [61]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3941 entries, 0 to 3940
Data columns (total 46 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   property_name           3941 non-null   object 
 1   property_type           3941 non-null   object 
 2   society                 3940 non-null   object 
 3   price                   3941 non-null   float64
 4   area                    3941 non-null   float64
 5   bedRoom                 3941 non-null   int64  
 6   bathroom                3941 non-null   int64  
 7   balcony                 3941 non-null   object 
 8   additionalRoom          3941 non-null   object 
 9   address                 3930 non-null   object 
 10  floorNum                3920 non-null   float64
 11  facing                  2770 non-null   object 
 12  airport                 3735 non-null   object 
 13  station                 3735 non-null   object 
 14  hospitals               3735 non-null   

In [62]:
# Society
data[data.society.isna()]

,property_name,property_type,society,price,area,bedRoom,bathroom,balcony,additionalRoom,address,floorNum,facing,airport,station,hospitals,banks,malls,educational_institutes,has_lift,has_swimming_pool,has_security_alarm,has_gym,has_park,has_parking,has_water_storage,status,avgRating,Modular Kitchen,Bed,Washing Machine,TV,Stove,Sofa,Fridge,Microwave,Dining Table,Water Purifier,Wardrobe,Curtains,Chimney,AC,Light,Geyser,Exhaust Fan,Fan,sector
2789,2 BHK Flat in Sector 78 Gurgaon,flat,NaN,0.6,1625.0,2,2,0,not available,"Gurgaon, Sector 78 Gurgaon, Gurgaon, Haryana",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,underConstruction,4.25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,78


In [63]:
# Too many nan values in this row
data.drop(2789, inplace=True)

In [64]:
data.isna().sum()

property_name                0
property_type                0
society                      0
price                        0
area                         0
bedRoom                      0
bathroom                     0
balcony                      0
additionalRoom               0
address                     11
floorNum                    20
facing                    1170
airport                    205
station                    205
hospitals                  205
banks                      205
malls                      205
educational_institutes     205
has_lift                   704
has_swimming_pool          704
has_security_alarm         704
has_gym                    704
has_park                   704
has_parking                704
has_water_storage          704
status                       0
avgRating                  448
Modular Kitchen           1026
Bed                       1026
Washing Machine           1026
TV                        1026
Stove                     1026
Sofa    

In [65]:
print(data.columns.values.tolist())

['property_name', 'property_type', 'society', 'price', 'area', 'bedRoom', 'bathroom', 'balcony', 'additionalRoom', 'address', 'floorNum', 'facing', 'airport', 'station', 'hospitals', 'banks', 'malls', 'educational_institutes', 'has_lift', 'has_swimming_pool', 'has_security_alarm', 'has_gym', 'has_park', 'has_parking', 'has_water_storage', 'status', 'avgRating', 'Modular Kitchen', 'Bed', 'Washing Machine', 'TV', 'Stove', 'Sofa', 'Fridge', 'Microwave', 'Dining Table', 'Water Purifier', 'Wardrobe', 'Curtains', 'Chimney', 'AC', 'Light', 'Geyser', 'Exhaust Fan', 'Fan', 'sector']


In [66]:
df = data[['property_type', 'price', 'area', 'bedRoom', 'bathroom', 'balcony', 'floorNum', 'facing', 'airport', 'station', 'hospitals', 'banks', 'malls', 'educational_institutes', 'has_lift', 'has_swimming_pool', 'has_security_alarm', 'has_gym', 'has_park', 'has_parking', 'has_water_storage', 'status', 'avgRating', 'Modular Kitchen', 'Bed', 'Washing Machine', 'TV', 'Stove', 'Sofa', 'Fridge', 'Microwave', 'Dining Table', 'Water Purifier', 'Wardrobe', 'Curtains', 'Chimney', 'AC', 'Light', 'Geyser', 'Exhaust Fan', 'Fan', 'sector']]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3940 entries, 0 to 3940
Data columns (total 42 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   property_type           3940 non-null   object 
 1   price                   3940 non-null   float64
 2   area                    3940 non-null   float64
 3   bedRoom                 3940 non-null   int64  
 4   bathroom                3940 non-null   int64  
 5   balcony                 3940 non-null   object 
 6   floorNum                3920 non-null   float64
 7   facing                  2770 non-null   object 
 8   airport                 3735 non-null   object 
 9   station                 3735 non-null   object 
 10  hospitals               3735 non-null   float64
 11  banks                   3735 non-null   float64
 12  malls                   3735 non-null   float64
 13  educational_institutes  3735 non-null   float64
 14  has_lift                3236 non-null   

In [67]:
df.sample(10)

,property_type,price,area,bedRoom,bathroom,balcony,floorNum,facing,airport,station,hospitals,banks,malls,educational_institutes,has_lift,has_swimming_pool,has_security_alarm,has_gym,has_park,has_parking,has_water_storage,status,avgRating,Modular Kitchen,Bed,Washing Machine,TV,Stove,Sofa,Fridge,Microwave,Dining Table,Water Purifier,Wardrobe,Curtains,Chimney,AC,Light,Geyser,Exhaust Fan,Fan,sector
3358,flat,1.32,1569.0,3,3,1,12.0,South-East,True,False,1.0,0.0,1.0,1.0,False,True,False,True,True,True,False,constructed,4.80,1,No,No,No,No,No,No,No,No,No,2,No,No,No,10,No,No,5,69
1830,flat,6.75,2704.0,4,4,2,8.0,North,False,True,3.0,1.0,0.0,4.0,False,True,True,True,True,True,True,constructed,5.00,1,No,1,No,1,No,1,1,No,1,6,No,1,6,10,4,1,6,54
311,flat,1.30,735.0,2,2,3,0.0,East,True,False,1.0,0.0,1.0,2.0,False,True,True,True,True,True,True,constructed,4.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102
1157,house,0.74,1000.0,3,3,0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,underConstruction,4.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105
361,flat,1.60,2545.0,3,3,3,9.0,North,False,True,9.0,0.0,0.0,0.0,False,True,False,True,True,True,True,constructed,4.75,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,109
3400,house,10.00,550.0,5,6,3,4.0,North-East,False,True,42.0,0.0,0.0,1.0,False,False,False,False,True,False,False,constructed,3.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33
3857,flat,1.15,1762.0,3,3,2,15.0,East,True,True,2.0,1.0,2.0,0.0,False,True,True,True,True,False,True,constructed,3.40,1,No,No,No,No,No,No,No,No,No,3,No,No,3,5,3,No,5,71
3582,flat,0.46,700.0,1,1,1,18.0,NaN,True,True,0.0,0.0,0.0,0.0,False,True,True,True,True,True,True,constructed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,106
3420,house,2.40,900.0,4,4,3,3.0,East,False,False,12.0,5.0,0.0,5.0,False,False,False,False,True,True,True,constructed,4.25,1,No,No,No,No,No,No,No,No,No,4,No,No,No,32,No,No,8,46
2618,flat,1.10,1295.0,3,3,2,2.0,North,True,False,1.0,0.0,1.0,2.0,False,False,True,False,True,True,False,underConstruction,4.00,No,No,No,No,No,No,No,No,No,No,1,No,1,3,6,No,No,3,37D


In [68]:
df.balcony.replace({'3+':4}, inplace=True)

<ipython-input-68-d23918d56c07>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.balcony.replace({'3+':4}, inplace=True)


In [69]:
df[df.isna().sum(axis=1)>20]

,property_type,price,area,bedRoom,bathroom,balcony,floorNum,facing,airport,station,hospitals,banks,malls,educational_institutes,has_lift,has_swimming_pool,has_security_alarm,has_gym,has_park,has_parking,has_water_storage,status,avgRating,Modular Kitchen,Bed,Washing Machine,TV,Stove,Sofa,Fridge,Microwave,Dining Table,Water Purifier,Wardrobe,Curtains,Chimney,AC,Light,Geyser,Exhaust Fan,Fan,sector
19,flat,1.15,1000.0,2,2,3,4.0,East,NaN,NaN,NaN,NaN,NaN,NaN,False,True,True,True,True,True,True,constructed,4.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70A
24,flat,1.20,1444.0,2,2,3,0.0,NaN,True,True,1.0,0.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,underConstruction,4.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
28,flat,0.90,1244.0,4,2,4,1.0,NaN,False,True,6.0,11.0,0.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,constructed,4.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
82,house,1.50,1980.0,7,2,0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,underConstruction,4.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
92,house,0.50,100.0,2,2,0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,underConstruction,4.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105
95,house,1.10,900.0,6,4,1,1.0,South-West,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,underConstruction,4.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105
98,house,2.80,1500.0,4,4,0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,underConstruction,4.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55
102,house,1.10,990.0,6,4,1,2.0,North,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,underConstruction,4.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
131,flat,1.83,1310.0,2,2,0,7.0,NaN,True,True,1.0,0.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,underConstruction,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,113
156,house,0.55,820.0,2,2,1,2.0,South-East,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,constructed,4.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3A


In [70]:


# Fill missing values in 'facing' column
data['facing'].fillna(data.groupby(['society', 'sector'])['facing'].transform('first'), inplace=True)

# Fill missing values in the specified columns
columns_to_fill = [
    'airport', 'station', 'hospitals', 'banks', 'malls', 'educational_institutes',
    'has_lift', 'has_swimming_pool', 'has_security_alarm', 'has_gym', 'has_park',
    'has_parking', 'has_water_storage'
]

for column in columns_to_fill:
    data[column].fillna(data.groupby(['society', 'sector'])[column].transform('first'), inplace=True)
data.isna().sum()

property_name                0
property_type                0
society                      0
price                        0
area                         0
bedRoom                      0
bathroom                     0
balcony                      0
additionalRoom               0
address                     11
floorNum                    20
facing                     469
airport                    122
station                    122
hospitals                  122
banks                      122
malls                      122
educational_institutes     122
has_lift                   386
has_swimming_pool          386
has_security_alarm         386
has_gym                    386
has_park                   386
has_parking                386
has_water_storage          386
status                       0
avgRating                  448
Modular Kitchen           1026
Bed                       1026
Washing Machine           1026
TV                        1026
Stove                     1026
Sofa    

In [71]:
data[~data.has_lift.isna()].isna().sum()

property_name               0
property_type               0
society                     0
price                       0
area                        0
bedRoom                     0
bathroom                    0
balcony                     0
additionalRoom              0
address                    10
floorNum                   11
facing                    213
airport                    43
station                    43
hospitals                  43
banks                      43
malls                      43
educational_institutes     43
has_lift                    0
has_swimming_pool           0
has_security_alarm          0
has_gym                     0
has_park                    0
has_parking                 0
has_water_storage           0
status                      0
avgRating                 344
Modular Kitchen           923
Bed                       923
Washing Machine           923
TV                        923
Stove                     923
Sofa                      923
Fridge    

In [72]:
# Create a dictionary to map property names to sectors
property_name_sector_mapping = data.groupby('property_name')['sector'].first().to_dict()

# Fill missing 'sector' values based on 'property_name' and 'society'
data['sector'].fillna(data['property_name'].map(property_name_sector_mapping), inplace=True)

# If 'society' is the same, fill the same 'sector'
data['sector'] = data.groupby('society')['sector'].transform('first')

In [73]:
data.sector.isna().sum()

175

In [74]:
link_clean = "https://docs.google.com/spreadsheets/d/e/2PACX-1vSkREARXr3RM36l--9AfgQfajeVoEOzwlJSkTwZGRLOygVgNZRcj2Gx_-HqCwqUb3P6SsVC__OHSjTM/pub?gid=64276078&single=true&output=csv"
dfk = pd.read_csv(link_clean)
dfk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10285 entries, 0 to 10284
Data columns (total 55 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   SPID                        10285 non-null  int64  
 1   PROP_ID                     10285 non-null  object 
 2   PREFERENCE                  10285 non-null  object 
 3   DESCRIPTION                 10285 non-null  object 
 4   PROPERTY_TYPE               10285 non-null  object 
 5   CITY                        10285 non-null  object 
 6   TRANSACT_TYPE               7498 non-null   float64
 7   OWNTYPE                     10285 non-null  int64  
 8   BEDROOM_NUM                 9273 non-null   float64
 9   PRICE_PER_UNIT_AREA         10285 non-null  int64  
 10  FURNISH                     10285 non-null  int64  
 11  FACING                      10285 non-null  int64  
 12  AGE                         10285 non-null  int64  
 13  TOTAL_FLOOR                 102

In [75]:
import ast
dfk['sector'] = dfk['location'].apply(lambda x: ast.literal_eval(x)['LOCALITY_NAME'])

In [76]:
dfk['PROP_HEADING'] = dfk['PROP_HEADING'].str.lower()
dfk['SOCIETY_NAME'] = dfk['SOCIETY_NAME'].str.lower()
data['property_name'] = data['property_name'].str.lower()

In [77]:
# Assuming 'dfk' is your new dataset
sector_mapping = dfk.groupby(['SOCIETY_NAME'])['sector'].first().to_dict()
# Assuming 'df' is your original dataset

# Create a function to fill missing 'sector' values using the mapping
def fill_sector(row):
    if pd.isna(row['sector']):
        key = row['society']
        # print(key)
        if key in sector_mapping:
            return sector_mapping[key]
    return row['sector']

# Apply the function to fill missing 'sector' values
data['sector'] = data.apply(fill_sector, axis=1)

In [78]:
len(sector_mapping)

944

In [79]:
data[~data.has_lift.isna()].isna().sum()

property_name               0
property_type               0
society                     0
price                       0
area                        0
bedRoom                     0
bathroom                    0
balcony                     0
additionalRoom              0
address                    10
floorNum                   11
facing                    213
airport                    43
station                    43
hospitals                  43
banks                      43
malls                      43
educational_institutes     43
has_lift                    0
has_swimming_pool           0
has_security_alarm          0
has_gym                     0
has_park                    0
has_parking                 0
has_water_storage           0
status                      0
avgRating                 344
Modular Kitchen           923
Bed                       923
Washing Machine           923
TV                        923
Stove                     923
Sofa                      923
Fridge    

In [80]:
data.isna().sum()

property_name                0
property_type                0
society                      0
price                        0
area                         0
bedRoom                      0
bathroom                     0
balcony                      0
additionalRoom               0
address                     11
floorNum                    20
facing                     469
airport                    122
station                    122
hospitals                  122
banks                      122
malls                      122
educational_institutes     122
has_lift                   386
has_swimming_pool          386
has_security_alarm         386
has_gym                    386
has_park                   386
has_parking                386
has_water_storage          386
status                       0
avgRating                  448
Modular Kitchen           1026
Bed                       1026
Washing Machine           1026
TV                        1026
Stove                     1026
Sofa    

In [81]:
data.shape

(3940, 46)

In [87]:
df_clean = data[~data.isna().any(axis=1)].reset_index(drop=True).copy()

In [107]:
df_clean.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2230 entries, 0 to 2229
Data columns (total 46 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   property_name           2230 non-null   object 
 1   property_type           2230 non-null   object 
 2   society                 2230 non-null   object 
 3   price                   2230 non-null   float64
 4   area                    2230 non-null   float64
 5   bedRoom                 2230 non-null   int64  
 6   bathroom                2230 non-null   int64  
 7   balcony                 2230 non-null   int64  
 8   address                 2230 non-null   object 
 9   floorNum                2230 non-null   float64
 10  facing                  2230 non-null   object 
 11  airport                 2230 non-null   int64  
 12  station                 2230 non-null   int64  
 13  hospitals               2230 non-null   float64
 14  banks                   2230 non-null   

In [92]:
df_clean['balcony'] = pd.to_numeric(df_clean['balcony'].replace({'3+':4}))

In [99]:
# Assuming your dataset is stored in a DataFrame called 'df'
df_clean['additional_room'] = pd.to_numeric(df_clean['additionalRoom'].apply(lambda x: 0 if x == 'not available' else x.count(',') + 1))
df_clean.drop('additionalRoom', axis=1, inplace=True)

In [105]:
df_clean.airport = df_clean.airport.astype('int')

In [106]:
df_clean.station=df_clean.station.astype('int')

In [111]:
print(df_clean.columns.values.tolist())

['property_name', 'property_type', 'society', 'price', 'area', 'bedRoom', 'bathroom', 'balcony', 'address', 'floorNum', 'facing', 'airport', 'station', 'hospitals', 'banks', 'malls', 'educational_institutes', 'has_lift', 'has_swimming_pool', 'has_security_alarm', 'has_gym', 'has_park', 'has_parking', 'has_water_storage', 'status', 'avgRating', 'Modular Kitchen', 'Bed', 'Washing Machine', 'TV', 'Stove', 'Sofa', 'Fridge', 'Microwave', 'Dining Table', 'Water Purifier', 'Wardrobe', 'Curtains', 'Chimney', 'AC', 'Light', 'Geyser', 'Exhaust Fan', 'Fan', 'sector', 'additional_room']


In [112]:
columns_to_convert = ['has_lift', 'has_swimming_pool', 'has_security_alarm', 'has_gym', 'has_park', 'has_parking', 'has_water_storage', 'Modular Kitchen', 'Bed', 'Washing Machine', 'TV', 'Stove', 'Sofa', 'Fridge', 'Microwave', 'Dining Table', 'Water Purifier', 'Wardrobe', 'Curtains', 'Chimney', 'AC', 'Light', 'Geyser', 'Exhaust Fan', 'Fan']

df_clean[columns_to_convert] = df_clean[columns_to_convert].apply(pd.to_numeric, errors='coerce')


In [113]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2230 entries, 0 to 2229
Data columns (total 46 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   property_name           2230 non-null   object 
 1   property_type           2230 non-null   object 
 2   society                 2230 non-null   object 
 3   price                   2230 non-null   float64
 4   area                    2230 non-null   float64
 5   bedRoom                 2230 non-null   int64  
 6   bathroom                2230 non-null   int64  
 7   balcony                 2230 non-null   int64  
 8   address                 2230 non-null   object 
 9   floorNum                2230 non-null   float64
 10  facing                  2230 non-null   object 
 11  airport                 2230 non-null   int64  
 12  station                 2230 non-null   int64  
 13  hospitals               2230 non-null   float64
 14  banks                   2230 non-null   